# Plotting Slope and Aspect (from QGIS)

In [ ]:
# Cell 1: imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cmcrameri.cm as cmr
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA


In [ ]:
# Cell 2: load Rolling‑Window sheets with “all” option
fp_r    = input("Enter Rolling‑Window Excel path: ")
all_r   = pd.read_excel(fp_r, sheet_name=None)
avail_r = list(all_r.keys())
sel_r   = input(f"Choose Rolling sheet(s) ({', '.join(avail_r)}) or 'all': ")
chosen_r = avail_r if sel_r.lower().strip()=="all" else [s.strip() for s in sel_r.split(",")]
df_r = {name: df.copy() for name, df in all_r.items() if name in chosen_r}
for df in df_r.values():
    df.columns = df.columns.str.strip().str.lower()


In [ ]:
# Cell 3: load Slope‑Aspect sheets with “all” option
fp_s    = input("Enter Slope‑Aspect Excel path: ")
all_s   = pd.read_excel(fp_s, sheet_name=None)
avail_s = list(all_s.keys())
sel_s   = input(f"Choose Slope-Aspect sheet(s) ({', '.join(avail_s)}) or 'all': ")
chosen_s = avail_s if sel_s.lower().strip()=="all" else [s.strip() for s in sel_s.split(",")]
df_s = {name: df.copy() for name, df in all_s.items() if name in chosen_s}
for df in df_s.values():
    df.columns = df.columns.str.strip().str.lower()
    df.rename(columns={"value":"z","slope_value":"slope","aspect_value":"aspect"}, inplace=True)


In [ ]:
# Cell 4: find matching sheets
common = [n for n in df_r if n in df_s]
if not common:
    raise ValueError("No overlapping sheets found")
print("Will plot:", common)


In [ ]:
# Cell 5: redefine plotting function with legend
def plot_sheet(name, dfr, dfs):
    s_r, dz, base, zdt, ribs = (
        dfr["distance"], dfr["z"], dfr["baseline"],
        dfr["z_detrended"], dfr["is_rib"].astype(bool)
    )
    s_s, slope, aspect = dfs["distance"], dfs["slope"], dfs["aspect"]
    fig = plt.figure(figsize=(8, fig_height))
    host = host_subplot(111, axes_class=AA.Axes); plt.subplots_adjust(right=0.75)
    par_s = host.twinx(); par_a = host.twinx(); par_a.spines["right"].set_position(("axes",1.2))
    # semi‑transparent curves behind
    par_s.plot(s_s, slope,   alpha=0.2, color=col_s, zorder=0, label="Slope")
    par_a.plot(s_s, aspect,  alpha=0.2, color=col_a, zorder=0, label="Aspect")
    # 3rd‑order solid trends
    ts = np.poly1d(np.polyfit(s_s, slope, 3))(s_s)
    ta = np.poly1d(np.polyfit(s_s, aspect,3))(s_s)
    par_s.plot(s_s, ts, color=col_s, zorder=1, label="Slope trend")
    par_a.plot(s_s, ta, color=col_a, zorder=1, label="Aspect trend")
    # profile & baseline
    host.plot(s_r, dz,    color="black", zorder=2, label="Profile")
    host.plot(s_r, base, "--", color=base_col, zorder=2, label="Baseline")
    # ribs behind profile
    host.scatter(
        s_r[ribs], base[ribs]+zdt[ribs],
        s=60, marker="o", facecolor=rib_col,
        edgecolor="none", zorder=1, label="Ribs"
    )
    # labels
    host.set_xlabel("Distance"); host.set_ylabel("Depth")
    par_s.set_ylabel("Slope");    par_a.set_ylabel("Aspect")
    # legend outside on right
    lines, labs = [], []
    for ax in (host, par_s, par_a):
        l, la = ax.get_legend_handles_labels(); lines += l; labs += la
    seen, hl, ll = set(), [], []
    for h, l in zip(lines, labs):
        if l not in seen:
            seen.add(l); hl.append(h); ll.append(l)
    host.legend(hl, ll, loc="center left", bbox_to_anchor=(1.02, 0.5), fontsize=8)
    # save
    out = f"{name}_slope.png"
    fig.savefig(out, dpi=300, bbox_inches="tight"); print(f"Saved to {out}")
    plt.show()


In [ ]:
# Cell 6: loop and auto‐save all sheets
for name in common:
    plot_sheet(name, df_r[name], df_s[name])


# Scatter plots
## Spacing vs slope

In [ ]:
# Cell 1: imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cmcrameri.cm as cmr
from scipy.spatial import cKDTree


In [ ]:
# Cell 2: load one or more Slope‑Aspect workbooks, keyed by sheet name
import pandas as pd
import cmcrameri.cm as cmr
from scipy.spatial import cKDTree

fps = input("Enter path(s) to Slope‑Aspect Excel file(s), comma-separated: ")
sheets_s = {}
for fp in [p.strip() for p in fps.split(",")]:
    wb = pd.read_excel(fp, sheet_name=None)
    for sheet, df in wb.items():
        df = df.copy()
        df.columns = df.columns.str.strip().str.lower()
        sheets_s[sheet] = df.rename(columns={
            "value": "z",
            "slope_value": "slope",
            "aspect_value": "aspect",
            "x": "x",
            "y": "y"
        })


In [ ]:
# Cell 3: load one or more Rib‑Details workbooks, keyed by sheet name
fps_r = input("Enter path(s) to Rib‑Details Excel file(s), comma-separated: ")
sheets_r = {}
for fp in [p.strip() for p in fps_r.split(",")]:
    wb = pd.read_excel(fp, sheet_name=None)
    for sheet, df in wb.items():
        df = df.copy()
        df.columns = df.columns.str.strip().str.lower()
        sheets_r[sheet] = df


In [ ]:
# Cell 4: merge each sheet (by nearest lat/lon) into one DataFrame
import numpy as np

merged = []
for name, sdf in sheets_s.items():
    if name not in sheets_r:
        continue
    rdf = sheets_r[name]
    # build KD tree on slope points
    tree = cKDTree(np.column_stack((sdf["x"], sdf["y"])))
    pts  = np.column_stack((rdf["longitude"], rdf["latitude"]))
    _, idx = tree.query(pts)
    merged.append(pd.DataFrame({
        "sheet":   name,
        "spacing": rdf["spacing"],
        "slope":   sdf["slope"].values[idx]
    }))

if not merged:
    raise ValueError("No matching sheets found between the two sets!")
dfm = pd.concat(merged, ignore_index=True)


In [ ]:
# Cell 5: assign group labels & colors
# group by first character
dfm["group"] = dfm["sheet"].str[0].apply(
    lambda c: f"Drumlin {c}" if c.isdigit() else "Drumlin Tails"
)
# sort groups: numbers ascending, then tails
num = sorted({g for g in dfm["group"] if g!="Drumlin Tails"},
             key=lambda s: int(s.split()[1]))
groups = num + (["Drumlin Tails"] if "Drumlin Tails" in dfm["group"].values else [])
colors = cmr.batlow(np.linspace(0.2, 0.8, len(groups)))
col_map = dict(zip(groups, colors))


In [ ]:
# Cell 6: scatter plot spacing vs slope colored by group
fig, ax = plt.subplots(figsize=(10, 6))
for grp in groups:
    sub = dfm[dfm["group"]==grp]
    ax.scatter(sub["spacing"], sub["slope"],
               color=col_map[grp], label=grp,
               s=40, edgecolor="k", alpha=0.7)
ax.set_xlabel("Spacing")
ax.set_ylabel("Slope")
ax.set_title("Slope vs Spacing by Drumlin Group")
ax.legend(title="Group", bbox_to_anchor=(1.02, 1),
          loc="upper left", fontsize="small")
plt.tight_layout()


In [ ]:
# Cell 7: save figure
out = input("Enter output filename (e.g. spacing_by_group.png): ")
fig.savefig(out, dpi=300, bbox_inches="tight")
print(f"Saved to {out}")
